# Get Active Fires Products in the California Based on Interagency Perimeters

## Setup Connection to Drive

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
rootPath = "/content/drive/My Drive/California FireTrends (2012-2020)"
os.chdir(rootPath)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!grep MemTotal /proc/meminfo

MemTotal:       26751732 kB


## Import Packages

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import rtree

In [ ]:
def listFiles_ByExt(rootPath, ext):
  '''
  retrieve file path + names based on extension
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
      for names in files: 
          if names.endswith(ext):
              file_list.append(os.path.join(path,names))
  return file_list

def createFolder(rootPath, folderName): 
  '''
  Create new folder in root path 
  '''
  folderPath = os.path.join(rootPath, folderName) 
  if not os.path.exists(folderPath):
      os.makedirs(folderPath)
  return folderPath 


def listFiles(rootPath):
  '''
  retrieve file path + names 
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
    for names in files: 
      file_list.append(os.path.join(path,names))
  return file_list

In [ ]:
# thomas = gpd.read_file('Products/By_Fire/2018/Thomas/Thomas_2018_NAT.shp')
# thomas = thomas.replace(regex='2018', value='2017')
# thomas = thomas.replace(2018, 2017)
# thomas.to_file('Products/By_Fire/2018/Thomas/Thomas_2018_NAT.shp', driver='ESRI Shapefile')

In [ ]:
# thomas.to_file('Products/By_Fire/2017/Thomas/Thomas_2017_NAT.geojson', driver='GeoJSON')
# thomas.to_file('Products/By_Fire/2017/Thomas/Thomas_2017_NAT.shp', driver='ESRI Shapefile')

In [ ]:
# thomas = gpd.read_file('Products/By_Fire/2018/Thomas/Thomas_2018_NAT.shp')
# thomas

## Interagency Perimeters

### Data Acquisition 

NearTime Perimeters (2020): 

https://opendata.arcgis.com/datasets/5da472c6d27b4b67970acc7b5044c862_0.geojson


Archived Wildfire Perimeters 2019-2020:

https://opendata.arcgis.com/datasets/bf373b4ff85e4f0299036ecc31a1bcbb_0.geojson

Historic Perimeters (2000-2018): 

https://opendata.arcgis.com/datasets/72a928a667634be1b795aa76a61e95f8_19.geojson


In [ ]:
outpth = 'Data/Interagency_FirePerimeters'
import urllib.request

url_dict = (('FirePerimeters_Current', 'https://opendata.arcgis.com/datasets/5da472c6d27b4b67970acc7b5044c862_0.geojson'),
         ('FirePerimeters_Archived', 'https://opendata.arcgis.com/datasets/bf373b4ff85e4f0299036ecc31a1bcbb_0.geojson'),
         ('FirePerimeters_2000_2018', 'https://opendata.arcgis.com/datasets/72a928a667634be1b795aa76a61e95f8_19.geojson'))

# Download the file from `url` and save it locally under `file_name`:
for file_nm, url in url_dict:
  urllib.request.urlretrieve(url, os.path.join(outpth,f'{file_nm}.geojson'))

### Add FRAP

In [ ]:
from urllib.request import urlretrieve

FRAP_perimeters = 'https://frap.fire.ca.gov/media/10969/fire19_1.zip'

urlretrieve(FRAP_perimeters, os.path.join(outpth, 'Fire_Perimeters_FRAP'))

('Data/Interagency_FirePerimeters/Fire_Perimeters_FRAP',
 <http.client.HTTPMessage at 0x7f91798f20f0>)

In [ ]:
import zipfile

with zipfile.ZipFile(os.path.join(outpth, 'Fire_Perimeters_FRAP'), 'r') as zip_ref:
    zip_ref.extractall(outpth)

In [ ]:
import fiona
outpth = 'Data/Interagency_FirePerimeters'
fiona.listlayers(os.path.join(outpth, 'fire19_1.gdb'))
FRAP_2019 = gpd.read_file(os.path.join(outpth, 'fire19_1.gdb'), layer='firep19_1')

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:548: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [ ]:
FRAP_2019 = FRAP_2019[(FRAP_2019['YEAR_'].notna()) & (FRAP_2019['YEAR_'] != "")]
FRAP_2019['YEAR_'].astype('int64').min(), FRAP_2019['YEAR_'].astype('int64').max()

(1878, 2019)

In [ ]:
# only look at 2012 to 2019 ; in CA 
# update column names to reflect same
# merge with geomac, remove duplicates 
FRAP_2019.columns

Index(['YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME', 'INC_NUM',
       'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')

In [ ]:
FRAP_2019 = FRAP_2019.rename(columns={'FIRE_NAME': 'IncidentName', 
                          'GIS_ACRES': 'GISAcres', 
                          'ALARM_DATE': 'ALARM_DATE', 
                          'CONT_DATE': 'CONT_DATE',
                          'complexname': 'ComplexName'})

In [ ]:
FRAP_2019['CONT_DATE'] = pd.to_datetime(FRAP_2019['CONT_DATE'], format='%Y-%m-%dT%H:%M:%S', errors = 'coerce')
FRAP_2019['ALARM_DATE'] = pd.to_datetime(FRAP_2019['ALARM_DATE'], format='%Y-%m-%dT%H:%M:%S', errors = 'coerce')
FRAP_2019['MapMethod'] = 'FRAP'

### Pre-processing 

In [ ]:
# access perimeter data in Drive
perimeter_path = 'Data/Interagency_FirePerimeters'

fire_per = listFiles_ByExt(perimeter_path, '.geojson')
fire_per

['Data/Interagency_FirePerimeters/FirePerimeters_Current.geojson',
 'Data/Interagency_FirePerimeters/FirePerimeters_Archived.geojson',
 'Data/Interagency_FirePerimeters/FirePerimeters_2000_2018.geojson',
 'Data/Interagency_FirePerimeters/WESTERN_US/Combined_2012_2020/Perimeters_2012_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Combined500_2012_2020/Perimeters500_2012_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Interagency_Full_Perimeters/Perimeters_2012_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Combined_2012_2020/Perimeters_2012_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Combined_2000_2020/Perimeters_2000_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Combined_2000_2020_Simplified/Simplified_Perimeters_2000_2020.geojson',
 'Data/Interagency_FirePerimeters/CALIFORNIA_updated_Oct2020/Combined_2000_2020_DissolvedbyYear/DissolvedbyYe

In [ ]:
outpth = 'Data/Interagency_FirePerimeters'
df_current = gpd.read_file(os.path.join(outpth,f'FirePerimeters_Current.geojson')) 
df2_archived = gpd.read_file(os.path.join(outpth,f'FirePerimeters_Archived.geojson')) 
df3_historical = gpd.read_file(os.path.join(outpth,f'FirePerimeters_2000_2018.geojson')) 

Add 'CONT_DATE' and 'ALARM_DATE'
- First combine all 

In [ ]:
def renameColumns(df, yr_range, mnth_range):
  '''
  make sure all columns name match 
  filter by column name, create datetime columns, filter by year range (2000 to 2020)
  '''
  df = df.rename(columns={'incidentname': 'IncidentName', 
                          'mapmethod': 'MapMethod', 
                          'gisacres': 'GISAcres', 
                          'perimeterdatetime': 'BurnDate', 
                          'CreateDate': 'BurnDate', 
                          'DateCurrent': 'LastUpdated',
                          'datecurrent': 'LastUpdated',
                          'complexname': 'ComplexName',
                          'irwinid': 'IRWINID'})
  if 'uniquefireidentifier' in df.columns:
    df['IRWINID'] = np.where(df['IRWINID'].isnull(),df['uniquefireidentifier'],df['IRWINID'])
  if 'GlobalID' in df.columns:
    df['IRWINID'] = np.where(df['IRWINID'].isnull(),df['GlobalID'],df['IRWINID'])
  # filter by column
  df = df[column_names]
  # create column for year, month, day 
  df['BurnDate']= pd.to_datetime(df['BurnDate'], format='%Y-%m-%d')
  df['Year'] = pd.DatetimeIndex(df['BurnDate']).year
  df['Month'] = pd.DatetimeIndex(df['BurnDate']).month
  df['Day'] = pd.DatetimeIndex(df['BurnDate']).day
  df['IncidentName'] = df['IncidentName'].str.title()
  # filter by year
  df = df[df.Year.isin(yr_range)]
  df = df[df.Month.isin(mnth_range)]
  return df

In [ ]:
column_names = ['IncidentName', 'MapMethod', 'GISAcres', 'BurnDate', 'LastUpdated', 'ComplexName', 'GlobalID', 'IRWINID', 'geometry']
yr_range = range(2000, 2021)
mnth_range = range(1, 12)

current = renameColumns(df_current, yr_range, mnth_range)
archived = renameColumns(df2_archived, yr_range, mnth_range)
df_frames = [current, archived]
recent_df = pd.concat(df_frames)

# remove dupes from current and archived sets 
# recent_df_filt = recent_df.groupby(['GlobalID','IncidentName', 'Year']).apply(lambda x: x.loc[x.CreateDate.idxmax()])

In [ ]:
column_names = ['IncidentName', 'MapMethod', 'GISAcres', 'BurnDate', 'LastUpdated', 'ComplexName', 'IRWINID', 'geometry']

historical = renameColumns(df3_historical, yr_range, mnth_range)

# remove dupes from historical set
# historical_filt = historical.groupby(['IRWINID','IncidentName', 'Year']).apply(lambda x: x.loc[x.GISAcres.idxmax()])

df_frames = [historical, recent_df]
complete_df = pd.concat(df_frames)

In [ ]:
alarmdt_df = complete_df[complete_df.groupby(['IRWINID','IncidentName', 'Year']).BurnDate.transform('min') == complete_df['BurnDate']]

In [ ]:
alarmdt_df['ALARM_DATE'] = alarmdt_df['BurnDate']
alarmdt_df = alarmdt_df[['IRWINID','IncidentName', 'Year', 'ALARM_DATE']]

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
complete_df = complete_df.merge(alarmdt_df, on = ['IRWINID','IncidentName', 'Year'])

In [ ]:
contdt_df = complete_df[complete_df.groupby(['IRWINID','IncidentName', 'Year']).BurnDate.transform('max') == complete_df['BurnDate']]

In [ ]:
contdt_df['CONT_DATE'] = contdt_df['BurnDate']
contdt_df = contdt_df[['IRWINID','IncidentName', 'Year', 'CONT_DATE']]

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
complete_df = complete_df.merge(contdt_df, on = ['IRWINID','IncidentName', 'Year'])

In [ ]:
complete_df = complete_df.sort_values(by=['GISAcres', 'IncidentName', 'Year'], ascending=False)
complete_df = complete_df.drop_duplicates(subset=['IncidentName', 'Year'], keep='first')

complete_df['IRWINID'] = complete_df['IRWINID'].str.lower()
complete_df['IRWINID'] = complete_df['IRWINID'].str.replace('{','')
complete_df['IRWINID'] = complete_df['IRWINID'].str.replace('}','')
complete_df = complete_df.drop_duplicates(subset=['IRWINID'], keep='first')

In [ ]:
complete_df = gpd.GeoDataFrame(complete_df, geometry=complete_df.geometry)
complete_df = complete_df.set_crs('EPSG:4326')
complete_df = complete_df.to_crs('EPSG:3310')

In [ ]:
complete_df.columns


Index(['IncidentName', 'MapMethod', 'GISAcres', 'BurnDate', 'LastUpdated',
       'ComplexName', 'IRWINID', 'geometry', 'Year', 'Month', 'Day',
       'GlobalID', 'ALARM_DATE', 'CONT_DATE'],
      dtype='object')

In [ ]:
FRAP_2019.columns

Index(['YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'IncidentName', 'INC_NUM',
       'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC', 'GISAcres',
       'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Length', 'Shape_Area',
       'geometry', 'MapMethod'],
      dtype='object')

In [ ]:
FRAP_2019['YEAR_'].astype('int64').min(), FRAP_2019['YEAR_'].astype('int64').max()

In [ ]:
column_names = ['IncidentName',  'MapMethod', 'GISAcres', 'ALARM_DATE', 'CONT_DATE', 'geometry']

FRAP_per = renameColumns(FRAP_2019, yr_range, mnth_range)

In [ ]:
FRAP_per.columns

Index(['IncidentName', 'MapMethod', 'GISAcres', 'ALARM_DATE', 'CONT_DATE',
       'geometry', 'Year', 'Month', 'Day'],
      dtype='object')

In [ ]:
complete_df.columns

Index(['IncidentName', 'MapMethod', 'GISAcres', 'BurnDate', 'LastUpdated',
       'ComplexName', 'IRWINID', 'geometry', 'Year', 'Month', 'Day',
       'GlobalID', 'ALARM_DATE', 'CONT_DATE'],
      dtype='object')

In [ ]:
df_frames = [complete_df, FRAP_per]
complete = pd.concat(df_frames)

In [ ]:
complete["Area"] = complete['geometry'].area * 0.0002471054

In [ ]:
complete = complete[['IncidentName', 'MapMethod', 'GISAcres', 'Area', 'CONT_DATE', 'Year', 'Month', 'Day', 'geometry']]
# complete['ALARM_DATE'] = complete['ALARM_DATE'].dt.strftime('%Y-%m-%d')
complete['CONT_DATE'] = complete['CONT_DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
# fire naming, to remove duplicate fire instances 
complete['Year']  = complete['Year'].astype('int64')
complete['IncidentName'] = complete['IncidentName'].str.strip()
complete['IncidentName'] = complete['IncidentName'].str.replace(' Fire','')
complete['IncidentName'] = complete['IncidentName'].str.replace('-','')
complete['IncidentName'] = complete['IncidentName'].str.replace('_',' ')
complete['IncidentName'] = complete['IncidentName'].str.replace(' Complex','')
complete['IncidentName'] = complete['IncidentName'].str.replace('.','')
complete = complete.sort_values(by=['Area', 'IncidentName', 'Year'], ascending=False)
complete = complete.drop_duplicates(subset=['IncidentName', 'Year'], keep='first')

In [ ]:
# save thus far as shp, geojson
path = 'Data/Interagency_FirePerimeters/WESTERN_US'
outpath = createFolder(path, 'Combined_2000_2020')

filename = f'{outpath}/Perimeters_2000_2020.shp'
complete.to_file(filename, driver='ESRI Shapefile')

filename = f'{outpath}/Perimeters_2000_2020.geojson'
complete.to_file(filename, driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [ ]:
state_path = '/content/drive/My Drive/COVID_FireTrends/Data/US_states/'
states = gpd.read_file(state_path)
CA_states = states[(states.STATE_NAME == 'California')]
CA_states = CA_states.to_crs('EPSG:3310')
CA_states_buff = CA_states
CA_states_buff['geometry'] = CA_states.geometry.buffer(100000)

In [ ]:
complete['geometry'] = complete.geometry.buffer(0)

In [ ]:
complete_CA = gpd.overlay(complete, CA_states_buff, how='intersection')

In [ ]:
complete_CA = complete_CA.sort_values(by=['Area', 'IncidentName', 'Year'], ascending=False)
complete_CA = complete_CA.drop_duplicates(subset=['IncidentName', 'Year'], keep='first')

In [ ]:
path = 'Data/Interagency_FirePerimeters/CALIFORNIA'
outpath = createFolder(path, 'Combined_2000_2020')

filename = f'{outpath}/Perimeters_2000_2020.shp'
complete_CA.to_file(filename, driver='ESRI Shapefile') #ESRI Shapefile

filename = f'{outpath}/Perimeters_2000_2020.geojson'
complete_CA.to_file(filename, driver='GeoJSON') 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """


In [ ]:
df_frames = [complete_df, FRAP_per]
all_df = pd.concat(df_frames)

In [ ]:
all_df = all_df[['Year', 'geometry']]

In [ ]:
all_df = gpd.GeoDataFrame(all_df, geometry='geometry')

In [ ]:
all_df.geometry = all_df.geometry.buffer(0)

In [ ]:
all_df['Year'] = all_df['Year'].astype('int64').astype('str')

In [ ]:
Annual_Dissolved = all_df.dissolve(by=['Year']).reset_index()

In [ ]:
Annual_Dissolved = Annual_Dissolved[['geometry']]
Annual_Dissolved_singlePoly = Annual_Dissolved.explode()
Annual_Dissolved_singlePoly['GIS_ACRES'] = Annual_Dissolved_singlePoly['geometry'].area * 0.0002471054
Annual_Dissolved_singlePoly[['Year', 'GIS_ACRES', 'geometry']]

In [ ]:
path = 'Data/Interagency_FirePerimeters/CALIFORNIA'
outpath = createFolder(path, 'Combined_2000_2020_DissolvedbyYear')

filename = f'{outpath}/DissolvedbyYear_Perimeters_2000_2020.shp'
Annual_Dissolved_singlePoly.to_file(filename, driver='ESRI Shapefile') #ESRI Shapefile

filename = f'{outpath}/DissolvedbyYear_Perimeters_2000_2020.geojson'
Annual_Dissolved_singlePoly.to_file(filename, driver='GeoJSON') 

In [ ]:
complete_CA_500 = complete_CA[complete_CA['Area'] >= 500]

In [ ]:
path = 'Data/Interagency_FirePerimeters/CALIFORNIA'
outpath = createFolder(path, 'Combined500_2012_2020')

filename = f'{outpath}/Perimeters500_2012_2020.shp'
complete_CA_500.to_file(filename, driver='ESRI Shapefile')

filename = f'{outpath}/Perimeters500_2012_2020.geojson'
complete_CA_500.to_file(filename, driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """


In [ ]:
complete_CA_500.groupby('Year')['IncidentName'].nunique()

Year
2012     72
2013     50
2014     59
2015     79
2016     63
2017    140
2018     86
2019     45
2020     86
Name: IncidentName, dtype: int64

In [ ]:
len(complete_CA_500)

680

## Active Fire Products


### Data Acquisition 

Archive Download: https://firms.modaps.eosdis.nasa.gov/download/


Near-RealTime (7 days) 

MODIS (C6): https://firms.modaps.eosdis.nasa.gov/data/active_fire/c6/shapes/zips/MODIS_C6_USA_contiguous_and_Hawaii_7d.zip

VIIRS (S-NPP): 
https://firms.modaps.eosdis.nasa.gov/data/active_fire/suomi-npp-viirs-c2/shapes/zips/SUOMI_VIIRS_C2_USA_contiguous_and_Hawaii_7d.zip

VIIRS (NOAA-20): Note: launched Nov 2017 
https://firms.modaps.eosdis.nasa.gov/data/active_fire/noaa-20-viirs-c2/shapes/zips/J1_VIIRS_C2_USA_contiguous_and_Hawaii_7d.zip

In [ ]:
from datetime import date
import urllib.request
import requests, zipfile, io

VIIRS_SPP = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/suomi-npp-viirs-c2/shapes/zips/SUOMI_VIIRS_C2_USA_contiguous_and_Hawaii_7d.zip'
VIIRS_NOAA20 = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/noaa-20-viirs-c2/shapes/zips/J1_VIIRS_C2_USA_contiguous_and_Hawaii_7d.zip'
MODIS_1km = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/c6/shapes/zips/MODIS_C6_USA_contiguous_and_Hawaii_7d.zip'

path = 'Data/Active_Fire_Products'
outpth = createFolder(path, 'Current')

# https://console.cloud.google.com/storage/browser/activefires_bucket
# gs://activefires_bucket

today = date.today()
d1 = today.strftime("%m%d%Y")

url_dict = ((f'suomi_viirs_7d_{d1}', VIIRS_SPP),
         (f'noaa_viirs_7d_{d1}', VIIRS_NOAA20),
          (f'aquaterra_modis_7d_{d1}', MODIS_1km))

# Download the file from `url` and save it locally under `file_name`:
for file_nm, url in url_dict:
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  createFolder(outpth, file_nm)
  z.extractall(os.path.join(outpth, file_nm))
  list_of_files = listFiles(os.path.join(outpth, file_nm))
  for fls in list_of_files:
    filename, file_extension = os.path.splitext(fls)
    os.rename(fls, os.path.join(outpth, file_nm, file_nm + file_extension))

### Pre-Processing

In [ ]:
AF_archive_path = 'Data/Active_Fire_Products/Archived'
af_archive = glob.glob(os.path.join(AF_archive_path, '*', '*.shp'))

In [ ]:
import glob, os
AF_nrt_path = 'Data/Active_Fire_Products/Current'
af_nrt = glob.glob(os.path.join(AF_nrt_path, '*', '*.shp'))

In [ ]:
aquaterra_list = [s for s in af_nrt if "aquaterra" in s]
noaaviirs_list = [s for s in af_nrt if "noaa_viirs" in s]
suomiviirs_list = [s for s in af_nrt if "suomi_viirs" in s]

In [ ]:
path = 'Data/Active_Fire_Products/Current_Merged'

def mergeMultipleSHP(shapefiles, path, compile_name):
  outpath = createFolder(path, compile_name)
  gdf = pd.concat([
      gpd.read_file(shp)
      for shp in shapefiles
  ]).pipe(gpd.GeoDataFrame)
  gdf.to_file(os.path.join(outpath,f'{compile_name}.shp'))

mergeMultipleSHP(aquaterra_list, path, 'AF_M6_C6_compiled')
mergeMultipleSHP(suomiviirs_list, path, 'AF_V1_C2_compiled')
mergeMultipleSHP(noaaviirs_list, path, 'AF_JV1_C2_compiled')

In [ ]:
# read af files 
AF_archive_path = 'Data/Active_Fire_Products/Archived'
AF_nrt_path = 'Data/Active_Fire_Products/Current_Merged'

# ARCHIVE
df1 = gpd.read_file(os.path.join(AF_archive_path,'DL_FIRE_M6_150883/fire_archive_M6_150883.shp'))
df2 = gpd.read_file(os.path.join(AF_archive_path,'DL_FIRE_M6_150883/fire_nrt_M6_150883.shp'))
df3 = gpd.read_file(os.path.join(AF_archive_path,'DL_FIRE_V1_150885/fire_archive_V1_150885.shp'))
df4 = gpd.read_file(os.path.join(AF_archive_path,'DL_FIRE_V1_150885/fire_nrt_V1_150885.shp'))

# NRT Merged Downloads
df5 = gpd.read_file(os.path.join(AF_nrt_path,'AF_M6_C6_compiled/AF_M6_C6_compiled.shp'))
df6 = gpd.read_file(os.path.join(AF_nrt_path,'AF_V1_C2_compiled/AF_V1_C2_compiled.shp'))

In [ ]:
#j1v_nrt, 
frames = [df1, df2, df3, df4, df5, df6]

In [ ]:
df6['INSTRUMENT'] = 'VIIRS'
df5['INSTRUMENT'] = 'MODIS'

In [ ]:
af_pts = ['Archive_M6', 'NRT_M6', 'Archive_V1', 'NRT_V1', 'Current_M6', 'Current_V1']

In [ ]:
def getFileInfo(dfFrames):
  '''
  From geodf in frames read from data folder with AF products, 
  create dictionary based on INSTRUMENT
  '''
  dic = {'MODIS': [], 'VIIRS': []}
  for i, df in enumerate(dfFrames): 
    minDate = min(df['ACQ_DATE'])
    maxDate = max(df['ACQ_DATE'])
    numEntries = len(df)
    print('filename: ', af_pts[i], '\nminDate: ', minDate, 
          '\nmaxDate: ',maxDate, '\nnumEntries: ',numEntries)
    x = 'df'+ str(i+1)
    if df['INSTRUMENT'][0] == 'MODIS': 
      dic['MODIS'].append(x)
    else: 
      dic['VIIRS'].append(x)
  return dic

In [ ]:
dic_frames = getFileInfo(frames)

filename:  Archive_M6 
minDate:  2012-01-01 
maxDate:  2020-06-30 
numEntries:  1032301
filename:  NRT_M6 
minDate:  2020-07-01 
maxDate:  2020-09-06 
numEntries:  27237
filename:  Archive_V1 
minDate:  2012-01-20 
maxDate:  2020-05-31 
numEntries:  4025286
filename:  NRT_V1 
minDate:  2020-06-01 
maxDate:  2020-09-06 
numEntries:  205364
filename:  Current_M6 
minDate:  2020-09-01 
maxDate:  2020-10-07 
numEntries:  93200
filename:  Current_V1 
minDate:  2020-09-01 
maxDate:  2020-10-07 
numEntries:  455337


In [ ]:
# combine based on sensor 
# MODIS AF (1km) 
MODIS_AF = pd.concat([eval(x) for x in dic_frames['MODIS']])
MODIS_AF = MODIS_AF.to_crs('EPSG:3310')

In [ ]:
# VIIRS AF 
VIIRS_AF = pd.concat([eval(x) for x in dic_frames['VIIRS']])
VIIRS_AF = VIIRS_AF.to_crs('EPSG:3310')

### Clip to CA

In [ ]:
# check; set CRS 
state_path = '/content/drive/My Drive/COVID_FireTrends/Data/US_states/'
states = gpd.read_file(state_path)
CA_states = states[(states.STATE_NAME == 'California')]
CA_states = CA_states.to_crs('EPSG:3310')
CA_states_buff = CA_states
CA_states_buff['geometry'] = CA_states.geometry.buffer(100000)

In [ ]:
## clip to california extent 
modis_375_CA= gpd.overlay(MODIS_AF, CA_states_buff, how='intersection')
viirs_375_CA= gpd.overlay(VIIRS_AF, CA_states_buff, how='intersection')

### Deal with Duplicates

In [ ]:
modis_375_CA = modis_375_CA.drop_duplicates(subset=['LATITUDE', 'LONGITUDE', 'ACQ_DATE', 'ACQ_TIME', 'INSTRUMENT', 'geometry'])
viirs_375_CA = viirs_375_CA.drop_duplicates(subset=['LATITUDE', 'LONGITUDE', 'ACQ_DATE', 'ACQ_TIME', 'INSTRUMENT', 'geometry'])

### Convert UTC
 to PST

In [ ]:
def convertUTMtoPST(df, dt_field, tm_field, dt_format):
  df['dt'] = df[dt_field] + ' ' + df[tm_field]
  df['dt']= pd.to_datetime(df['dt'], format= dt_format)
  df['DATE_PST'] = df['dt'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
  df['dt'] = df['dt'].dt.strftime(dt_format)
  return df

def createDateFields(df, dt_field, dt_format):
  ''' from datetime field, creates 4 new columns: 
  Year, Month, Day, Julian Day
  '''
  df[dt_field]= pd.to_datetime(df[dt_field], format= dt_format)
  df['Year'] = pd.DatetimeIndex(df[dt_field]).year
  df['Month'] = pd.DatetimeIndex(df[dt_field]).month
  df['Day'] = pd.DatetimeIndex(df[dt_field]).day
  df['JD'] = df[dt_field].dt.strftime('%y%j').astype('int64')
  df['Date'] = df[dt_field].dt.strftime('%Y-%m-%d')
  df['Time'] = df[dt_field].dt.strftime('%H%M')
  df[dt_field] = df[dt_field].dt.strftime(dt_format)
  return df

In [ ]:
modis_375_CA = convertUTMtoPST(modis_375_CA, 'ACQ_DATE', 'ACQ_TIME', '%Y-%m-%d %H%M')
modis_375_CA  = createDateFields(modis_375_CA , 'DATE_PST', '%Y-%m-%d %H%M')

In [ ]:
viirs_375_CA = convertUTMtoPST(viirs_375_CA, 'ACQ_DATE', 'ACQ_TIME', '%Y-%m-%d %H%M')
viirs_375_CA = createDateFields(viirs_375_CA, 'DATE_PST', '%Y-%m-%d')

In [ ]:
AF_path = 'Data/Active_Fire_Products'

outpath = createFolder(AF_path, 'Filtered')

v_outpath = createFolder(outpath, 'VIIRS_375m')
m_outpath = createFolder(outpath, 'MODIS_1km')

viirs_375_CA.to_file(os.path.join(v_outpath, 'VIIRS_375m_2012_2020.shp'))
modis_375_CA.to_file(os.path.join(m_outpath, 'MODIS_1km_2012_2020.shp'))

## Resampling MODIS

### Create individual layer for each day of M6 for resampling

In [ ]:
m_outpath = createFolder(outpath, 'MODIS_1km')
modis_375_CA = gpd.read_file(os.path.join(m_outpath, 'MODIS_1km_2012_2020.shp'))

In [ ]:
df = modis_375_CA[modis_375_CA['Year'] == 2020]
df

In [ ]:
# create shp for each day in yr 
rootpath = 'Data/MODIS_Resampling/SHP'

for yr in range(2020, 2020 + 1):
  #create folder by yr
  outpath = createFolder(rootpath, str(yr))
  df = modis_375_CA[modis_375_CA['Year'] == yr]
  for jd in df['JD'].unique().tolist():
    day_path = createFolder(outpath, str(jd)[-3:])
    if os.path.exists(os.path.join(day_path, 'M6_' + str(yr) + "_" + str(jd)[-3:] + '.shp')) == False:
      daily_df = df[df['JD'] == jd]
      daily_df = daily_df.to_crs("EPSG:3310")
      daily_df.to_file(os.path.join(day_path, 'M6_' + str(yr) + "_" + str(jd)[-3:] + '.shp'))
      print('M6_' + str(yr) + "_" + str(jd)[-3:] + '.shp', 'created..')
    else: 
      continue

M6_2020_260.shp created..
M6_2020_261.shp created..
M6_2020_262.shp created..
M6_2020_263.shp created..
M6_2020_264.shp created..
M6_2020_265.shp created..
M6_2020_266.shp created..
M6_2020_267.shp created..
M6_2020_268.shp created..
M6_2020_269.shp created..
M6_2020_270.shp created..
M6_2020_271.shp created..
M6_2020_272.shp created..


In [ ]:
day_lst = modis_375_CA['JD'].unique().tolist()
len(day_lst)
# def rasMod(MODISAF):
#   '''
#   resample MODIS AF points to 375m by rasterizing 
#   '''
#   for day in day_lst:
#     afLayer = MODIS_AF_EE.filter(ee.Filter.equals('JD', day))
#     ras = afLayer.reduceToImage(['JD'], ee.Reducer.first()).reproject(crs='epsg:4326', scale=1000)
#     region = MODIS_AF_EE.geometry().bounds()
#     ras_375 = ras.resample('bicubic').reproject(crs='epsg:4326', scale=375)
#     pt_375 = ras_375.reduceToVectors(**{'geometry': region, 'maxPixels':45135575, 'scale': 375,
#                                     'geometryType': 'centroid','labelProperty': 'JD'})

2917

### Resampling MODIS to 375m done using arcpy package and merged into 'Data/Active_Fire_Products/Filtered/MODIS_375m'

In [ ]:
! pip install geopandas

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
resamp_m6_files = listFiles_ByExt('Data/MODIS_Resampling/RESAMP_SHP_DT', 'shp')

In [ ]:
len(resamp_m6_files)

2905

In [ ]:
# merge all daily resampled MODIS points into one file
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in resamp_m6_files
]).pipe(gpd.GeoDataFrame)
gdf.to_file(os.path.join('Data/MODIS_Resampling/RESAMP_SHP_ALL','MODIS_375m_2012_2020.shp'))

In [ ]:
# import os
# import shutil

# source = r'Data/MODIS_Resampling/RESAMP_SHP_ALL'
# destination = m_outpath = createFolder('Data/Active_Fire_Products/Filtered', 'MODIS_375m')

# src_files = os.listdir(source)
# for file_name in src_files:
#     full_file_name = os.path.join(source, file_name)
#     if os.path.isfile(full_file_name):
#         shutil.copy(full_file_name, destination)

In [ ]:
mod_375 = gpd.read_file('Data/MODIS_Resampling/RESAMP_SHP_ALL/MODIS_375m_2012_2020.shp')

In [ ]:
mod_375 = mod_375.drop(columns=['Join_Count', 'TARGET_FID', 'pointid', 'pointid_1', 'grid_cod_1'])
mod_375['grid_code'] = mod_375['grid_code'].astype('int64').astype('str')

In [ ]:
def createDateFields(df, dt_field, dt_format):
  ''' from datetime field, creates 4 new columns: 
  Year, Month, Day, Julian Day
  '''
  df[dt_field]= pd.to_datetime(df[dt_field], format= dt_format)
  df['Year'] = pd.DatetimeIndex(df[dt_field]).year
  df['Month'] = pd.DatetimeIndex(df[dt_field]).month
  df['Day'] = pd.DatetimeIndex(df[dt_field]).day
  df['JD'] = df[dt_field].dt.strftime('%y%j').astype('int64')
  df['Date'] = df[dt_field].dt.strftime('%Y-%m-%d')
  df[dt_field] = df[dt_field].dt.strftime(dt_format)
  return df

In [ ]:
# date columns 
mod_375_new = createDateFields(mod_375, 'grid_code', "%y%j")

In [ ]:
mod_375_new['INSTRUMENT'] = 'MODIS'

In [ ]:
outpath = 'Data/Active_Fire_Products/Filtered'
m3_outpath = createFolder(outpath, 'MODIS_375m')
#mod_375_new['grid_code'] = mod_375_new['grid_code'].dt.strftime('%Y-%m-%d')
mod_375_new.to_file(os.path.join(m3_outpath, 'MODIS_375m_2012_2020.shp'))

## Merge VIIRS & resampled M6 

In [ ]:
outpath = 'Data/Active_Fire_Products/Filtered'
modis_375 = gpd.read_file(os.path.join(outpath, 'MODIS_375m', 'MODIS_375m_2012_2020.shp'))
viirs_375 = gpd.read_file(os.path.join(outpath, 'VIIRS_375m', 'VIIRS_375m_2012_2020.shp'))

In [ ]:
modis_375 = modis_375.to_crs('EPSG:4326')
modis_375['LONGITUDE'] = modis_375['geometry'].x
modis_375['LATITUDE'] = modis_375['geometry'].y
modis_375

,grid_code,Time,Year,Month,Day,JD,Date,INSTRUMENT,geometry,LONGITUDE,LATITUDE
0,12125,1450,2012,5,4,12125,2012-05-04,MODIS,POINT (-122.17533 39.35195),-122.175334,39.351948
1,12125,1450,2012,5,4,12125,2012-05-04,MODIS,POINT (-122.17098 39.35203),-122.170981,39.352025
2,12125,1450,2012,5,4,12125,2012-05-04,MODIS,POINT (-122.16663 39.35210),-122.166627,39.352102
3,12125,1450,2012,5,4,12125,2012-05-04,MODIS,POINT (-122.17523 39.34857),-122.175234,39.348574
4,12125,1450,2012,5,4,12125,2012-05-04,MODIS,POINT (-122.17088 39.34865),-122.170881,39.348651
...,...,...,...,...,...,...,...,...,...,...,...
880200,19296,1358,2019,10,23,19296,2019-10-23,MODIS,POINT (-114.45045 32.74212),-114.450448,32.742125
880201,19296,1358,2019,10,23,19296,2019-10-23,MODIS,POINT (-114.44646 32.74193),-114.446460,32.741926
880202,19296,1358,2019,10,23,19296,2019-10-23,MODIS,POINT (-114.45467 32.73894),-114.454671,32.738942
880203,19296,1358,2019,10,23,19296,2019-10-23,MODIS,POINT (-114.45068 32.73874),-114.450682,32.738744


In [ ]:
# # check; set CRS 
# state_path = 'Data/US_states/'
# states = gpd.read_file(state_path)
# CA_states = states[(states.STATE_NAME == 'California')]
# CA_states = CA_states.to_crs('EPSG:3310')
# CA_states_buff = CA_states
# CA_states_buff['geometry'] = CA_states.geometry.buffer(100000)

In [ ]:
modis_375 = modis_375.to_crs('EPSG:3310')
viirs_375 = viirs_375.to_crs('EPSG:3310')

In [ ]:
# ## clip to california extent 
# modis_375_CA= gpd.overlay(w_modis_375, CA_states_buff, how='intersection')
# viirs_375_CA= gpd.overlay(w_viirs_375, CA_states_buff, how='intersection')

In [ ]:
# outpath = 'Data/Active_Fire_Products/NEW/Filtered_California'
# m3_outpath = createFolder(outpath, 'MODIS_375m')

# modis_375_CA.to_file(os.path.join(m3_outpath, 'MODIS_375m_2012_2020.shp'))

In [ ]:
# v3_outpath = createFolder(outpath, 'VIIRS_375m')

# viirs_375_CA.to_file(os.path.join(v3_outpath, 'VIIRS_375m_2012_2020.shp'))

In [ ]:
# Merge 
df_frames = [modis_375, viirs_375]
CA_AF_DF = pd.concat(df_frames)

In [ ]:
CA_AF_DF = CA_AF_DF[['Year', 'Month', 'Day', 'JD', 'Date', 'Time', 
       'LONGITUDE', 'LATITUDE', 'INSTRUMENT','geometry']]

#CA_AF_DF['Date'] = CA_AF_DF['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
merged_outpath = createFolder(outpath, 'Merged_AF')

CA_AF_DF.to_file(os.path.join(merged_outpath, 'Merged_375m_2012_2020.shp'))

In [ ]:
CA_AF_DF.to_file(os.path.join(merged_outpath, 'Merged_375m_2012_2020.geojson'), driver='GeoJSON')

## Clip by Fire instance within date range & save

In [ ]:
complete_CA_500 = gpd.read_file('Data/Interagency_FirePerimeters/CALIFORNIA/Combined500_2012_2020/Perimeters500_2012_2020.geojson')

In [ ]:
complete_CA_500['Year']  = complete_CA_500['Year'].astype('int64')
complete_CA_500['IncidentName'] = complete_CA_500['IncidentName'].str.strip()
complete_CA_500 = complete_CA_500.sort_values(by=['Area', 'IncidentName', 'Year'], ascending=False)
complete_CA_500 = complete_CA_500.drop_duplicates(subset=['IncidentName', 'Year'], keep='first')

In [ ]:
complete_CA_500 = complete_CA_500.reset_index()

In [ ]:
complete_CA_500.groupby('Year')['IncidentName'].nunique()

Year
2012     72
2013     50
2014     59
2015     79
2016     63
2017    140
2018     86
2019     45
2020     86
Name: IncidentName, dtype: int64

In [ ]:
outpath = 'Data/Active_Fire_Products/Filtered'
merged_outpath = createFolder(outpath, 'Merged_AF')
CA_AF_DF = gpd.read_file(os.path.join(merged_outpath, 'Merged_375m_2012_2020.geojson'))

In [ ]:
CA_AF_DF['Date'] =  pd.to_datetime(CA_AF_DF['Date'], format='%Y-%m-%d')

In [ ]:
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

outpath = 'Data/FireInstances'

for i, row in complete_CA_500.iterrows():
  if os.path.exists(os.path.join(outpath, str(row.Year), row.IncidentName, f'{row.IncidentName}_{str(row.Year)}_MergedAF.shp')) == False:
    per_dt = datetime.datetime.strptime(row.CreateDate, '%Y-%m-%d')
    end_dt = per_dt + relativedelta(weeks=+2)
    start_dt = per_dt + relativedelta(months=-2)

    AF_filt_df = CA_AF_DF[(CA_AF_DF['Date'] >= start_dt) & (CA_AF_DF['Date'] <= end_dt)]

    per_gdf = complete_CA_500[i: i + 1] 
    per_gdf['geometry'] = complete_CA_500[i : i + 1].geometry.buffer(2000)

    AF_perFire= gpd.overlay(AF_filt_df, per_gdf, how='intersection')

    
    if len(AF_perFire) > 0: 
      yr_folder = createFolder(outpath, str(row.Year))
      af_outpath = createFolder(yr_folder, row.IncidentName)

      AF_perFire['Date'] = AF_perFire['Date'].dt.strftime('%Y-%m-%d')
      AF_perFire['Fire'] = row.IncidentName

      AF_perFire['JulianDay'] = AF_perFire['JD'].astype(str).str[-3:].astype('int64')
      AF_perFiƒbare = AF_perFire.drop(columns = ['Year_2', 'Month_2', 'Day_2'])
      AF_perFire = AF_perFire.rename(columns={"Year_1": "Year", "Month_1": "Month",'Day_1': 'Day'})

      AF_perFire.to_file(os.path.join(af_outpath, f'{row.IncidentName}_{str(row.Year)}_MergedAF.shp'))
      print(i, row['IncidentName'], per_dt, start_dt, end_dt)
    else: 
      print(i, row['IncidentName'], 'EMPTY') 
  else: 
    print(f'{row.IncidentName}_{str(row.Year)}_MergedAF.shp', 'exists')

In [ ]:
outpath = 'Data/FireInstances'
len(listFiles_ByExt(outpath, '.shp'))

714

## Interpolate using ArcPY

## Clean-up Interpolated Surfaces 
- Add in attributes (datetime, firename, year, etc) 

In [ ]:
def createDateFields(df, dt_field, dt_format):
  ''' from datetime field, creates 4 new columns: 
  Year, Month, Day, Julian Day
  '''
  df[dt_field]= pd.to_datetime(df[dt_field], format= dt_format)
  df['Year'] = pd.DatetimeIndex(df[dt_field]).year
  df['Month'] = pd.DatetimeIndex(df[dt_field]).month
  df['Day'] = pd.DatetimeIndex(df[dt_field]).day
  df['JD'] = df[dt_field].dt.strftime('%y%j').astype('int64')
  df['Date'] = df[dt_field].dt.strftime('%Y-%m-%d')
  df[dt_field] = df[dt_field].dt.strftime(dt_format)
  return df

In [ ]:
outpath = 'Data/Interpolation/SHP/2020'
pth_lst = listFiles_ByExt(outpath, '.shp')

In [ ]:
len(pth_lst)

83

In [ ]:
!ls Products/By_Fire/

2012  2013  2014  2015	2016  2017  2018  2019	2020_0916_2020


In [ ]:
for inter_fr in pth_lst:
  file_nm = os.path.basename(inter_fr)

  fire = inter_fr.split('/')[4]
  year = file_nm.split('_')[1]

  outpth = inter_fr.replace('Data','Products')
  outpth = outpth.replace(file_nm,'')
  outpth = outpth.replace('SHP/','')
  outpth = outpth.replace('Interpolation/','By_Fire/')
  file_jsn = file_nm.replace('.shp', '.geojson')

  if os.path.exists(os.path.join(outpth, file_jsn)) == False: 
    try: 
      AF_path = f'Data/FireInstances/{year}/{fire}/{fire}_{year}_MergedAF.shp'
      AF_gpd = gpd.read_file(AF_path)
      AF_Time = AF_gpd.groupby(['JD'])['Time'].agg(['min', 'max']).reset_index()
      AF_Time.columns = ['JulianDay', 'Time_Min', 'Time_Max']
      AF_Time['JulianDay'] = AF_Time['JulianDay'].astype(str).str[-3:].astype('int64')

      orig = gpd.read_file(inter_fr)
      df = orig.dissolve(by='gridcode').reset_index()
      df['YRJD'] = year + df['gridcode'].astype(str)
      df['Fire'] = fire
      
      new_df = createDateFields(df, 'YRJD', '%Y%j')
      new_df = new_df.reset_index().drop(columns=['index', 'Id', 'JD'])
      new_df = new_df.rename(columns={'gridcode': 'JulianDay'})
      new_df["Area (acres)"] = new_df.geometry.area * 0.0002471054
      new_df["Area (ha)"] = new_df.geometry.area * 0.0001
      final_df = new_df.merge(AF_Time, on=['JulianDay'])

      if not os.path.exists(outpth):
          os.makedirs(outpth)

      final_df.to_file(os.path.join(outpth, file_jsn), driver='GeoJSON')
      final_df.to_file(os.path.join(outpth, file_nm))
      print(os.path.join(outpth, file_jsn))
    except: 
      print('error', os.path.join(outpth, file_jsn))

In [ ]:
import glob, os
#merge by year 
outpath = 'Products/By_Year'
files_path = 'Products/By_Fire'
pth_lst = glob.glob(os.path.join(files_path, '*', '*', '*.geojson'))

for yr in range(2020, 2020 + 1):
  print(yr)
  yr = str(yr)
  yr_lst = [s for s in pth_lst if yr in s]
  yrpath = createFolder(outpath, yr)
  gdf = pd.concat([
      gpd.read_file(shp)
      for shp in yr_lst
      ]).pipe(gpd.GeoDataFrame)
  gdf.to_file(os.path.join(yrpath, f'{yr}_DailyFirePerimeters.shp'))
  gdf.to_file(os.path.join(yrpath, f'{yr}_DailyFirePerimeters.geojson'), driver='GeoJSON')
  print('complete')

2020


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  app.launch_new_instance()


complete


In [ ]:
rootPath = os.path.join(r'Products', 'By_Fire')
inter_files = listFiles_ByExt(rootPath,'.geojson')

In [ ]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')

gdf = pd.concat([
    gpd.read_file(shp)
    for shp in inter_files
    ]).pipe(gpd.GeoDataFrame)

gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


## Create Ignition layer from Incident start dates

Uses DBSCAN clustering technique to derive ignition instances for earliest detected datetime. (previously wanted to incorporate FRAPS incident Alarm date, but was not entirely accurate?) 

For each clustering, create envelope/convex hull and get centroid based on group_id's. 

In [ ]:
# import fiona
# outpth = 'Data/Interagency_FirePerimeters'
# fiona.listlayers(os.path.join(outpth, 'fire19_1.gdb'))
# FRAP_2019 = gpd.read_file(os.path.join(outpth, 'fire19_1.gdb'), layer='firep19_1')

In [ ]:
# FRAP_2019 = FRAP_2019[(FRAP_2019['YEAR_'].notna()) & (FRAP_2019['YEAR_'] != "")]
# FRAP_2019['YEAR_'].astype('int64').min(), FRAP_2019['YEAR_'].astype('int64').max()

(1878, 2019)

In [ ]:
# df = FRAP_2019[['YEAR_', 'FIRE_NAME', 'ALARM_DATE',	'CONT_DATE', 'geometry']]
# df['FIRE_NAME'] = df['FIRE_NAME'].str.title()
# df['FIRE_NAME'] = df['FIRE_NAME'].str.strip()
# df['FIRE_NAME'] = df['FIRE_NAME'].str.replace(' Fire','')
# df['FIRE_NAME'] = df['FIRE_NAME'].str.replace('-','')
# df['FIRE_NAME'] = df['FIRE_NAME'].str.replace('_',' ')
# df['FIRE_NAME'] = df['FIRE_NAME'].str.replace(' Complex','')
# df['FIRE_NAME'] = df['FIRE_NAME'].str.replace('.','')
# df['YEAR_'] = df['YEAR_'].astype('int64')
# df = df[df['YEAR_'] >= 2012]
# df["Area"] = df['geometry'].area * 0.0002471054
# df = df[df['Area'] >= 500]
# df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'], format='%Y-%m-%dT%H:%M:%S', errors = 'coerce')
# df['ALARM_DATE'] = pd.to_datetime(df['ALARM_DATE'], format='%Y-%m-%dT%H:%M:%S', errors = 'coerce')
# #df.to_csv('fire_daf_Files = lstFiles('Data/FireInstances', 'shp')ates.csv')

In [ ]:
af_Files = lstFiles('Data/FireInstances', 'shp')
af_Files

In [ ]:
import datetime
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint

# yr = '2020'
# af_Files = ['Data/FireInstances/2020/Hennessey/Hennessey_2020_MergedAF.shp',
#             'Data/FireInstances/2018/Carr/Carr_2018_MergedAF.shp']
# yr_lst = [s for s in af_Files if yr in s]

ignition_dt = [] 
fire_nm = [] 
yr_nm = [] 
pth_nm = [] 

for AF in af_Files: 
  afnm = os.path.basename(AF)
  year = AF.split('/')[2]
  fire = AF.split('/')[3]
  print(year, fire, afnm)
  AF_DF = gpd.read_file(AF)

  AF_min_dt = datetime.datetime.strptime(AF_DF['Date'].min(), '%Y-%m-%d')

  ignition_dt.append(AF_min_dt)
  fire_nm.append(fire)
  yr_nm.append(year)
  pth_nm.append(AF)

  # ignition_points = AF_DF[AF_DF['Date']== AF_DF['Date'].min()]

  start_dt = AF_DF[AF_DF['Date']== AF_DF['Date'].min()]
  ignition_points = start_dt[start_dt['Time']== start_dt['Time'].min()]
  
  # DBSCAN clustering 
  # find multiple ignition points 
  epsilon = 0.03
  coords = ignition_points[['LONGITUDE', 'LATITUDE']].values
  db = (DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric = 'euclidean')
        .fit(coords))
  cluster_labels = db.labels_
  group_id = cluster_labels.tolist()
  ignition_points['group_id'] = group_id
  id_list = list(set(group_id))

  if not os.path.exists(f'Ignition/AF_Clusters/{year}/{fire}'):
      os.makedirs(f'Ignition/AF_Clusters/{year}/{fire}')
  
  firenmm = fire.strip()

  ignition_points.to_file(f'Ignition/AF_Clusters/{year}/{fire}/{firenmm}_{year}_IgnitionClusters.shp')

  fr = [] 
  yr = [] 
  centroid = []
  dt = [] 
  tm = [] 
  ignition_id = []

  for id in id_list: 
    ignition_cluster = ignition_points[ignition_points['group_id']== id]
    pnts_coord = ignition_cluster[['LONGITUDE', 'LATITUDE']].values
    points = MultiPoint(pnts_coord)
    
    centroid.append(points.representative_point())
    fr.append(fire)
    yr.append(year)
    dt.append(ignition_cluster['Date'].min())
    tm.append(ignition_cluster['Time'].min())
    ignition_id.append(id)
  
  ignition_df = pd.DataFrame({'fire_nm': fr, 'yr_nm': yr,
                            'Date': dt, 'Time': tm, 'group_id': ignition_id, 
                            'geometry': centroid}) 
  
  ignition_gdf = gpd.GeoDataFrame(ignition_df, geometry= ignition_df.geometry)

  if not os.path.exists(f'Ignition/Ignition_Points/{year}/{fire}'):
      os.makedirs(f'Ignition/Ignition_Points/{year}/{fire}')

  ignition_gdf = ignition_gdf.set_crs('EPSG:4326')
  ignition_gdf = ignition_gdf.to_crs('EPSG:3310')

  
  ignition_gdf.to_file(f'Ignition/Ignition_Points/{year}/{fire}/{firenmm}_{year}_IgnitionPnts.shp', driver='ESRI Shapefile')
 

2020 Hennessey Hennessey_2020_MergedAF.shp
2018 Carr Carr_2018_MergedAF.shp


/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:849: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
ignition_dates_df = pd.DataFrame({'fire_nm':fire_nm, 'yr_nm':yr_nm, 'pth_nm':pth_nm,'ignition_dt':ignition_dt}) 
ignition_dates_df.to_csv('fire_ignitiondates.csv')

In [ ]:
ignition_points.crs